In [1]:
#pip install plotly

In [2]:
#pip install keras

In [3]:
#pip install tensorflow

In [4]:
#pip install mkl

In [5]:
#pip install -c intel mkl

In [6]:
# lib para modelagem de dados 
import pandas as pd

In [7]:
# lib para uso de vetores 
import numpy as np 

In [8]:
# lib para funcoes matematicas 
import math

In [9]:
# lib para visualização gráfica 
import plotly.graph_objects as Dash

In [ ]:
# Libs para uso de machine learing do keras
from keras.models import Sequential

In [ ]:
from keras.layers import Dense, LSTM

In [ ]:
# lib para pre-processamento
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Lendo a base de dados 
base_dados = pd.read_csv('/home/whitehat000142/Documents/Repositorios/Projetos-ML/Dados/TSLA.')

In [ ]:
# Verificando os primeiros registros 
base_dados.head()

In [ ]:
# definir a coluna data como index do dataset
base_dados = base_dados.set_index('Date')

# retirando qualquer valor nulo existente nos dados
base_dados = base_dados.dropna()

# verificando
base_dados.head()


In [ ]:
# plotar um gráfico dinâmico 
# no gráfico é possível filtrar pela legenda a informação 
# utilizar zooms

# definindo uma figura 
figura = Dash.figure()

# incluindo o eixo no gráfico - abertura 
figura.add_trace(Dash.Scatter(x=base_dados.index,y=base_dados.Open,
                             mode='lines',name='abertura',marker_color='#FF7F0E',
                             visible='legendonly'))

# incluindo o eixo no gráfico - maior 
figura.add_trace(Dash.Scatter(x=base_dados.index,y=base_dados.High,
                             mode='lines',name='maior',marker_color='#2CA02C',
                             visible='legendonly'))

# incluindo o eixo no gráfico - menor 
figura.add_trace(Dash.Scatter(x=base_dados.index,y=base_dados.Low,
                             mode='lines',name='menor',marker_color='#D62728',
                             visible='legendonly'))

# Incluindo o eixo no gráfico - fechamento 
figura.add_trace(Dash.Scatter(x=base_dados.index, y=base_dados.Close,
                             mode='lines',name='fechamento',marker_color='#1F77B4'))

# modificando o layout do grafico 
figura.update_layout(title='histórico de preço',titlefont_size=28,
                    xaxis=dict(title='historico de preco', titlefont_size=16,
                              tickfont_size=14), height=500,
                     yaxis=dict(title='preço em rupia indiana',titlefont_size=16,
                               tickfont_size=14),
                     legend=dict(y=1,x=1,bgcolor='rgba(255,255,255,0)',
                                bordercolor='rgba(255,255,255,0)'))

# mostrando o gráfico
figura.show()


In [ ]:
# filtrando os dados de fechamento 
dados_fechamento = base_dados.filter(['Close'])
# filtrando apenas os valores
dados_fechamento_valores=dados_fechamento.values
# arredondando o numero para cima usando o 'math.ceil'
dados_fechamento_valores_tamanho = math.ceil(len(dados_fechamento_valores)*0.8)

dados_fechamento_valores_tamanho

In [ ]:
# # aplicando escalonamento nos dados 
#definindo os parametros da funcao escalonamento
funcao_escalonamento_01 = MinMaxScaler()
# aplicando nos dados de fechamento 
dados_escalonados_fechamento = funcao_escalonamento_01.fit_transform(dados_fechamento_valores)
dados_escalonados_fechamento

In [ ]:
# definindo os dados de treinamento 
dados_treino = dados_escalonados_fechamento

# listas para receber os dados 
x_treinamento = []
y_treinamento = []

# loop para separar os dados de treino e teste 
# nesse loop vamos separar os dados em blocos de 60 valores 
for loop in range(60, len(dados_treino)):
    #separando os dados de treinamento x 
    filtrando_amostra_treinamento_x = dados_treino[loop-60:loop,0]
    x_treinamento.append(filtrando_amostra_treinamento_x)
    #separando os dados de treinamento y
    filtrando_amostra_treinamento_y = dados_treino[loop,0]
    y_treinamento.append(filtrando_amostra_treinamento_y)

In [ ]:
#transformando listas em um array
x_treinamento, y_treinamento = np.array(x_treinamento),np.array(y_treinamento)
# convertendo array para matriz 
x_treinamento = np.reshape(x_treinamento,(x_treinamento.shape[0],x_treinamento.shape[1],1))
# verificando a emissão da nossa matriz 
x_treinamento.shape

In [ ]:
# Definindo a funcao keras
# Essa funcao é uma pilha linear de camadas do keras 
modelo = Sequential()

# Adicionando as camadas e parametros para nossa rede neural
# Treinamento da rede neural recorrente 

# LSTM - Long Short Term Memory
modelo.add(LSTM(50,return_sequences=True,input_shape=(x_treinamento.shape[1],1)))
modelo.add(LSTM(50,return_sequences=False))

# adicionando as camadas na rede neural 
modelo.add(Dense(25))
modelo.add(Dense(1))
modelo.fit(x_treinamento,y_treinamento,batch_size=1,epochs=1)

In [ ]:
# definindo amostra para ser testada 
dados_teste = dados_escalonados_fechamento[dados_fechamento_valores_tamanho-60:,:]
# lista para receber os dados de teste 
x_teste = []
# lista com os dados de teste 
y_teste = dados_fechamento_valores[dados_fechamento_valores_tamanho,:]

# loop para fixar amostra para teste 
for loop in range(60, len(dados_teste)):
    x_teste.append(dados_teste[loop-60:loop,0])

# transformando os dados em um array 
x_teste = np.array(x_teste)
# convertendo o array para matriz 
x_teste = np.reshape(x_teste,(x_teste.shape[0],x_teste.shape[1],1))

# aplicando as previsões 
previsoes = modelo.predict(x_teste)
# invertendo para escalas reais 
previsoes = funcao_escalonamento_01.inverse_transform(previsoes)

# calculando o erro quadratico medio 
rsme = np.sqrt(np.mean(previsoes-y_teste)**2)
print(f'Erro quadratico medio:{rsme}')


In [ ]:
# atribuindo as previsoes no dataset
validacao['previsoes'] = previsoes


In [ ]:
# verificando o real vs modelo
validacao['Close','previsoes']].head()

In [ ]:
# criando um grafico dinamico
# no grafico é possivel filtrar pela legenda a informaçao 
# utilizar zoom

# definindo uma figura
figura = Dash.Figure()

# incluindo o eixo no grafico - fechamento 
figura.add_trace(Dash.Scatter(x=validacao.index,y=validacao.Close,mode='lines',
                             name='fechamento',marker_color='#FF7F0E',))

# incluindo o eixo no grafico - previsao 
figura.add_trace(Dash.Scatter(x=validacao.index,y=validacao.previsoes,mode='lines',
                             name='previsao',marker_color='#2CA02C',))

# Modificando o layout do gráfico 
figura.update_layout(title='realizado vs modelo',titlefont_size=28,
                     xaxis=dict(title='periodo historico',titlefont_size=16,
                                tickfont_size=14), height=500,
                     yaxis=dict(title='preço em rupia indiana',
                               titlefont_size=16,tickfont_size=14),
                    legend=dict(y=1,x=1,bgcolor='rgba(255,255,255,0)',
                               bordercolor='rgba(255,255,255,0)'))

figura.show()

